<a href="https://colab.research.google.com/github/41371103hjnh/114-1-/blob/main/HW1%E6%97%A5%E5%B8%B8%E6%94%AF%E5%87%BA%E9%80%9F%E7%AE%97%E8%88%87%E5%88%86%E6%94%A4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#HW1
拉麵店簡易點餐系統，
結帳時會附上今日運勢。

In [9]:
# 匯入需要的模組
from datetime import datetime
import random
import gspread
from google.colab import auth
from google.auth import default
import pytz

In [10]:
# Google Colab 認證
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

In [12]:
# 設定時區為台灣時間 (Asia/Taipei)
taiwan_tz = pytz.timezone('Asia/Taipei')

# 定義菜單
menu = {
    "1": {"name": "醬油拉麵", "price": 130},
    "2": {"name": "味噌拉麵", "price": 150},
    "3": {"name": "豚骨拉麵", "price": 150},
    "4": {"name": "麻辣拉麵", "price": 160},
    "5": {"name": "柚香雞白湯拉麵", "price": 230}
}

# 定義運勢列表
fortune_list = [
    "大吉：運勢爆表!順利到起飛啦~",
    "吉：吉吉富吉吉!",
    "中吉：勇於嘗試會有意想不到的收穫！",
    "小吉：平淡而美好的一天~",
    "末吉：莫寂寞莫慌害怕~",
    "凶：別讓挫折打倒你!",
    "大凶：來碗拉麵將逢凶化吉!"
]

# 顯示歡迎訊息與菜單
print("=====歡迎光臨柯基拉麵=====")
print("品項：")
for key in menu:
    print(f"{key}.{menu[key]['name']} {menu[key]['price']}元")
print("=========================")

# 使用者輸入日期與時間
date_input = input("請輸入日期 (YYYY-MM-DD) 或按Enter使用今天日期: ")
time_input = input("請輸入時間 (HH:MM) 或按Enter使用現在時間: ")

# 如果使用者沒輸入，就用系統時間（台灣時間）
now_taiwan = datetime.now(taiwan_tz)
date = date_input if date_input.strip() != "" else now_taiwan.strftime("%Y-%m-%d")
time = time_input if time_input.strip() != "" else now_taiwan.strftime("%H:%M")

# 初始化點餐清單與總金額
orders = []
total_price = 0

# 點餐迴圈
while True:
    item = input("請輸入品項編號 (空白結束): ")
    if item.strip() == "":
        break
    if item not in menu:
        print("輸入錯誤，請重新輸入")
        continue
    quantity = input("請輸入數量: ")
    if not quantity.isdigit() or int(quantity) <= 0:
        print("數量需為正整數，請重新輸入")
        continue
    quantity = int(quantity)
    # 加入訂單
    orders.append({"name": menu[item]["name"], "quantity": quantity, "price": menu[item]["price"]})
    total_price += menu[item]["price"] * quantity

# 顯示訂單明細
print("\n=====您的餐點如下=====")
print(f"日期: {date}  時間: {time}")
print("品項\t數量\t單價\t小計")
for order in orders:
    subtotal = order["price"] * order["quantity"]
    print(f"{order['name']}\t{order['quantity']}\t{order['price']}\t{subtotal}")

print(f"總金額: {total_price}元")

# 顯示隨機運勢
fortune = random.choice(fortune_list)
print(f"今日運勢：{fortune}")

print("=====謝謝您的光臨=====")

# 將訂單寫入 Google 試算表
try:
    # 打開您指定的試算表
    gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1K4Mz6mjHyLJJXb7PzLZ79kBxbW58xs4rIZ4_ZalPZj4/edit?usp=sharing')
    # 選擇要寫入的工作表
    worksheet = gsheets.worksheet('工作表1')

    # 遍歷訂單，並在最後一項加入總金額
    for i, order in enumerate(orders):
        is_last_item = (i == len(orders) - 1)
        subtotal = order["price"] * order["quantity"]

        row_data = [
            date,
            time,
            order["name"],
            order["quantity"],
            order["price"],
            subtotal,
            total_price if is_last_item else ""  # 只有在最後一筆時才顯示總金額
        ]
        # 在工作表的最後一列插入新資料
        worksheet.append_row(row_data)

    print("\n✅ 訂單資料已成功寫入 Google 試算表！")

except gspread.exceptions.APIError as e:
    print(f"\n❌ 寫入 Google 試算表時發生錯誤：{e}")
    print("請檢查您的試算表權限或網址是否正確。")

=====歡迎光臨柯基拉麵=====
品項：
1.醬油拉麵 130元
2.味噌拉麵 150元
3.豚骨拉麵 150元
4.麻辣拉麵 160元
5.柚香雞白湯拉麵 230元
請輸入日期 (YYYY-MM-DD) 或按Enter使用今天日期: 
請輸入時間 (HH:MM) 或按Enter使用現在時間: 
請輸入品項編號 (空白結束): 4
請輸入數量: 1
請輸入品項編號 (空白結束): 3
請輸入數量: 2
請輸入品項編號 (空白結束): 

=====您的餐點如下=====
日期: 2025-09-11  時間: 16:21
品項	數量	單價	小計
麻辣拉麵	1	160	160
豚骨拉麵	2	150	300
總金額: 460元
今日運勢：凶：別讓挫折打倒你!
=====謝謝您的光臨=====

✅ 訂單資料已成功寫入 Google 試算表！
